In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import requests
import re

In [2]:
# File 1 (7,429 entries)
#Museums by Discipline ART, BOT, CMU, HST, NAT, SCI, and ZAW
f1 = pd.read_csv('MuseumFile2018_File1_Nulls.csv', encoding = "ISO-8859-1")
print(f1.shape)
print(f1.columns.get_values())

(7431, 57)
['MID' 'DISCIPL' 'EIN' 'CO_LOC_RES' 'NTEEC' 'DUNS' 'NAICS' 'COMMONNAME'
 'LEGALNAME' 'ALTNAME' 'AKADBA' 'ADSTREET' 'ADCITY' 'ADSTATE' 'ADZIP'
 'ADZIP5' 'PHSTREET' 'PHCITY' 'PHSTATE' 'PHZIP' 'PHZIP5' 'PHONE' 'WEBURL'
 'IRS_NAME15' 'TAXPER15' 'INCOMECD15' 'INCOME15' 'REVENUE15'
 'IRS_STREET15' 'IRS_CITY15' 'IRS_STATE15' 'IRS_ZIP15' 'RULEDATE15'
 'IPEDS' 'INSTNAME' 'INSTCITY' 'INSTST' 'LONGITUDE' 'LATITUDE' 'AAMREG'
 'BEAREG' 'LOCALE4' 'FIPSST' 'FIPSCO' 'GSTREET' 'GCITY' 'GSTATE' 'GZIP'
 'GZIP5' 'BMF15_F' 'IMLSAD_F' 'PFND_F' 'FCT3P_F' 'USER_F' 'UNI_F'
 'OTHER_F' 'SOURCE']


In [ ]:
# File 2 (7,959 entries): General Museums and Related Organizations GMU (i.e., general museums)
f2 = pd.read_csv('MuseumFile2018_File2_Nulls.csv', encoding = "ISO-8859-1")
print(f2.shape)

In [ ]:
# File 3 (14,783 entries): Historical Societies/Historic Preservation HSC
f3 = pd.read_csv('MuseumFile2018_File3_Nulls.csv', encoding = "ISO-8859-1")
print(f3.shape)
# has extra empty row (row 0)
f3.drop(0, inplace=True)
print(f3.shape)

In [ ]:
fig1 = f1['DISCIPL'].value_counts().plot(kind="bar")
fig1.set_xlabel("Discipline")
fig1.set_ylabel("Count")
fig1.set_title("Museum Survey Disciplines")

In [ ]:
# Check state distribution of museums
print(f1['PHSTATE'].value_counts())
fig2 = f1['PHSTATE'].value_counts().plot(kind="bar", figsize = (12,5))
fig2.set_xlabel("State")
fig2.set_ylabel("Count")
fig2.set_title("Museum Survey States Distribution")


In [ ]:
#print(f1['PHSTATE'].head())
#print(f1['PHZIP'].head())
print(f1.columns.get_values())
#print(f1['COMMONNAME'].head())
print(f1[['LEGALNAME','PHSTREET','PHCITY','PHSTATE','PHZIP','PHZIP5', 'INSTST','LONGITUDE','LATITUDE']].head())

In [42]:
f_check = f1[f1['PHSTATE'] == ' ']
print(f_check.shape)
f_check[['LEGALNAME','PHSTREET','PHCITY','PHSTATE','PHZIP','PHZIP5', 'INSTST','LONGITUDE','LATITUDE']]


(5366, 57)


,LEGALNAME,PHSTREET,PHCITY,PHSTATE,PHZIP,PHZIP5,INSTST,LONGITUDE,LATITUDE
2,MOBILE MEDICAL MUSEUM INC,,,,,,,-88.07704,30.69234
3,JASMINE HILL GARDENS AND OUTDOOR MUSEUM,,,,,,,-86.1874,32.49562
4,JAN DEMPSEY ARTS CENTER,,,,,,,-85.4782,32.61408
5,HUNTSVILLE BOTANICAL GARDEN,,,,,,,-86.61819,34.71282
8,ALABAMA FOLK ART MUSEUM,,,,,,,-86.80921,33.51275
10,BARE HANDS GALLERY INC,,,,,,,-86.80581,33.51634
11,BELLINGRATH GARDENS AND HOME FOUNDATION INC,,,,,,,-88.15263,30.4637
12,RED BAY MUSEUM,,,,,,,-88.14745,34.44257
13,PIONEER MUSEUM OF ALABAMA ASSN,,,,,,,-85.99719,31.85412
14,BIRMINGHAM MUSEUM OF ART MEMBERS,,,,,,,-86.80939,33.52206


In [6]:
# Use reverse geocoding to use latitude and longitude to get the corresponding address components
api_key = ''
with open("geocoding_api_key.txt", 'r') as f:
    api_key = f.read()
    
f.close()
base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'

In [55]:
latlong_lookup =  f_check.iloc[0]['LATITUDE'] + ',' + f_check.iloc[0]['LONGITUDE']

In [84]:
parameters = {'latlng':latlong_lookup, 'key':api_key}
response = requests.get(base_url, params=parameters)

In [74]:
full_address = results['formatted_address']

address_components = full_address.split(',')
street = address_components[0]
city = address_components[1]
state_zip = address_components[2]

# extract state and zipcode
state = re.search('[A-Z]{2}', state_zip)[0]
zipcode_5 = re.search('\\d{5}', state_zip)[0]

'1310 Spring Hill Ave, Mobile, AL 36604, USA'